# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use.

### Tokenization

In the second cell, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
# open text file and read in data as `text`
with open('lstm_data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch:ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [5]:
int2char.items()

dict_items([(0, 'h'), (1, 'i'), (2, 'u'), (3, ';'), (4, 'a'), (5, 'P'), (6, 'y'), (7, 'G'), (8, 'r'), (9, 'T'), (10, 'x'), (11, 'Y'), (12, 'w'), (13, 'Z'), (14, '0'), (15, 'A'), (16, '7'), (17, 'J'), (18, 'H'), (19, '1'), (20, '('), (21, '"'), (22, 'k'), (23, 'V'), (24, 'v'), (25, 'C'), (26, 'o'), (27, ','), (28, 'D'), (29, 'M'), (30, '&'), (31, '/'), (32, 'f'), (33, '*'), (34, 'K'), (35, 'c'), (36, 'p'), (37, '!'), (38, 'F'), (39, 'm'), (40, 'l'), (41, 'W'), (42, 'E'), (43, 'L'), (44, '@'), (45, 'X'), (46, 's'), (47, 'O'), (48, '\n'), (49, 'Q'), (50, ':'), (51, '5'), (52, 'N'), (53, 'I'), (54, 'n'), (55, '-'), (56, '3'), (57, '%'), (58, '.'), (59, '?'), (60, 'U'), (61, 'q'), (62, 'e'), (63, '$'), (64, 'R'), (65, '4'), (66, "'"), (67, 'z'), (68, '6'), (69, ' '), (70, 'g'), (71, '_'), (72, 'S'), (73, '8'), (74, 'j'), (75, '`'), (76, 'b'), (77, 'd'), (78, ')'), (79, 't'), (80, '2'), (81, 'B'), (82, '9')])

In [6]:
encoded[:100]

array([25,  0,  4, 36, 79, 62,  8, 69, 19, 48, 48, 48, 18,  4, 36, 36,  6,
       69, 32,  4, 39,  1, 40,  1, 62, 46, 69,  4,  8, 62, 69,  4, 40, 40,
       69,  4, 40,  1, 22, 62,  3, 69, 62, 24, 62,  8,  6, 69,  2, 54,  0,
        4, 36, 36,  6, 69, 32,  4, 39,  1, 40,  6, 69,  1, 46, 69,  2, 54,
        0,  4, 36, 36,  6, 69,  1, 54, 69,  1, 79, 46, 69, 26, 12, 54, 48,
       12,  4,  6, 58, 48, 48, 42, 24, 62,  8,  6, 79,  0,  1, 54])

In [7]:
N_seqs = 10
N_steps = 50

# Get the number of characters per batch
batch_size = N_seqs * N_steps

## TODO: Get the number of batches we can make
n_batches = len(encoded) // batch_size

## TODO: Keep only enough characters to make full batches
ARR = encoded[0: N_seqs * N_steps * n_batches]

## TODO: Reshape into batch_size rows
ARR = ARR.reshape((N_seqs, -1))

# np.where(ARR == ARR[2])[1]

ARR.shape

(10, 198500)

In [8]:

## TODO: Make batches
for n in range(0, ARR.shape[1], N_steps):
    # The features
    x = ARR[:, n:n +N_steps]

    y = np.zeros_like(x)
    try:
        y[:, :-1], y[:, -1] = x[:, 1:], ARR[:, n+N_steps]
    except IndexError:
        y[:, :-1], y[:, -1] = x[:, 1:], ARR[:, 0]

In [9]:
nnn = 0

NEW_ARR = ARR[:, nnn:nnn +N_steps]
NEW_ARR

array([[25,  0,  4, 36, 79, 62,  8, 69, 19, 48, 48, 48, 18,  4, 36, 36,
         6, 69, 32,  4, 39,  1, 40,  1, 62, 46, 69,  4,  8, 62, 69,  4,
        40, 40, 69,  4, 40,  1, 22, 62,  3, 69, 62, 24, 62,  8,  6, 69,
         2, 54],
       [69,  4, 39, 69, 54, 26, 79, 69, 70, 26,  1, 54, 70, 69, 79, 26,
        69, 46, 79,  4,  6, 27, 21, 69,  4, 54, 46, 12, 62,  8, 62, 77,
        69, 15, 54, 54,  4, 27, 69, 46, 39,  1, 40,  1, 54, 70, 27, 69,
        76,  2],
       [24,  1, 54, 58, 48, 48, 21, 11, 62, 46, 27, 69,  1, 79, 66, 46,
        69, 46, 62, 79, 79, 40, 62, 77, 58, 69,  9,  0, 62, 69, 36,  8,
         1, 35, 62, 69,  1, 46, 69, 39,  4, 70, 54,  1, 32,  1, 35, 62,
        54, 79],
       [54, 69, 77,  2,  8,  1, 54, 70, 69,  0,  1, 46, 69, 35, 26, 54,
        24, 62,  8, 46,  4, 79,  1, 26, 54, 69, 12,  1, 79,  0, 69,  0,
         1, 46, 48, 76,  8, 26, 79,  0, 62,  8, 69, 12,  4, 46, 69, 79,
         0,  1],
       [69,  1, 79, 69,  1, 46, 27, 69, 46,  1,  8, 37, 21, 69, 46, 

In [10]:
y = np.zeros_like(NEW_ARR)

y[:,:-1] = NEW_ARR[:,1:]
y

array([[ 0,  4, 36, 79, 62,  8, 69, 19, 48, 48, 48, 18,  4, 36, 36,  6,
        69, 32,  4, 39,  1, 40,  1, 62, 46, 69,  4,  8, 62, 69,  4, 40,
        40, 69,  4, 40,  1, 22, 62,  3, 69, 62, 24, 62,  8,  6, 69,  2,
        54,  0],
       [ 4, 39, 69, 54, 26, 79, 69, 70, 26,  1, 54, 70, 69, 79, 26, 69,
        46, 79,  4,  6, 27, 21, 69,  4, 54, 46, 12, 62,  8, 62, 77, 69,
        15, 54, 54,  4, 27, 69, 46, 39,  1, 40,  1, 54, 70, 27, 69, 76,
         2,  0],
       [ 1, 54, 58, 48, 48, 21, 11, 62, 46, 27, 69,  1, 79, 66, 46, 69,
        46, 62, 79, 79, 40, 62, 77, 58, 69,  9,  0, 62, 69, 36,  8,  1,
        35, 62, 69,  1, 46, 69, 39,  4, 70, 54,  1, 32,  1, 35, 62, 54,
        79,  0],
       [69, 77,  2,  8,  1, 54, 70, 69,  0,  1, 46, 69, 35, 26, 54, 24,
        62,  8, 46,  4, 79,  1, 26, 54, 69, 12,  1, 79,  0, 69,  0,  1,
        46, 48, 76,  8, 26, 79,  0, 62,  8, 69, 12,  4, 46, 69, 79,  0,
         1,  0],
       [ 1, 79, 69,  1, 46, 27, 69, 46,  1,  8, 37, 21, 69, 46,  4, 

In [11]:
y[:, -1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
y[:, -1] = ARR[:, 0]
y

array([[ 0,  4, 36, 79, 62,  8, 69, 19, 48, 48, 48, 18,  4, 36, 36,  6,
        69, 32,  4, 39,  1, 40,  1, 62, 46, 69,  4,  8, 62, 69,  4, 40,
        40, 69,  4, 40,  1, 22, 62,  3, 69, 62, 24, 62,  8,  6, 69,  2,
        54, 25],
       [ 4, 39, 69, 54, 26, 79, 69, 70, 26,  1, 54, 70, 69, 79, 26, 69,
        46, 79,  4,  6, 27, 21, 69,  4, 54, 46, 12, 62,  8, 62, 77, 69,
        15, 54, 54,  4, 27, 69, 46, 39,  1, 40,  1, 54, 70, 27, 69, 76,
         2, 69],
       [ 1, 54, 58, 48, 48, 21, 11, 62, 46, 27, 69,  1, 79, 66, 46, 69,
        46, 62, 79, 79, 40, 62, 77, 58, 69,  9,  0, 62, 69, 36,  8,  1,
        35, 62, 69,  1, 46, 69, 39,  4, 70, 54,  1, 32,  1, 35, 62, 54,
        79, 24],
       [69, 77,  2,  8,  1, 54, 70, 69,  0,  1, 46, 69, 35, 26, 54, 24,
        62,  8, 46,  4, 79,  1, 26, 54, 69, 12,  1, 79,  0, 69,  0,  1,
        46, 48, 76,  8, 26, 79,  0, 62,  8, 69, 12,  4, 46, 69, 79,  0,
         1, 54],
       [ 1, 79, 69,  1, 46, 27, 69, 46,  1,  8, 37, 21, 69, 46,  4, 

In [13]:
int2char

{0: 'h',
 1: 'i',
 2: 'u',
 3: ';',
 4: 'a',
 5: 'P',
 6: 'y',
 7: 'G',
 8: 'r',
 9: 'T',
 10: 'x',
 11: 'Y',
 12: 'w',
 13: 'Z',
 14: '0',
 15: 'A',
 16: '7',
 17: 'J',
 18: 'H',
 19: '1',
 20: '(',
 21: '"',
 22: 'k',
 23: 'V',
 24: 'v',
 25: 'C',
 26: 'o',
 27: ',',
 28: 'D',
 29: 'M',
 30: '&',
 31: '/',
 32: 'f',
 33: '*',
 34: 'K',
 35: 'c',
 36: 'p',
 37: '!',
 38: 'F',
 39: 'm',
 40: 'l',
 41: 'W',
 42: 'E',
 43: 'L',
 44: '@',
 45: 'X',
 46: 's',
 47: 'O',
 48: '\n',
 49: 'Q',
 50: ':',
 51: '5',
 52: 'N',
 53: 'I',
 54: 'n',
 55: '-',
 56: '3',
 57: '%',
 58: '.',
 59: '?',
 60: 'U',
 61: 'q',
 62: 'e',
 63: '$',
 64: 'R',
 65: '4',
 66: "'",
 67: 'z',
 68: '6',
 69: ' ',
 70: 'g',
 71: '_',
 72: 'S',
 73: '8',
 74: 'j',
 75: '`',
 76: 'b',
 77: 'd',
 78: ')',
 79: 't',
 80: '2',
 81: 'B',
 82: '9'}

In [14]:
chars

('h',
 'i',
 'u',
 ';',
 'a',
 'P',
 'y',
 'G',
 'r',
 'T',
 'x',
 'Y',
 'w',
 'Z',
 '0',
 'A',
 '7',
 'J',
 'H',
 '1',
 '(',
 '"',
 'k',
 'V',
 'v',
 'C',
 'o',
 ',',
 'D',
 'M',
 '&',
 '/',
 'f',
 '*',
 'K',
 'c',
 'p',
 '!',
 'F',
 'm',
 'l',
 'W',
 'E',
 'L',
 '@',
 'X',
 's',
 'O',
 '\n',
 'Q',
 ':',
 '5',
 'N',
 'I',
 'n',
 '-',
 '3',
 '%',
 '.',
 '?',
 'U',
 'q',
 'e',
 '$',
 'R',
 '4',
 "'",
 'z',
 '6',
 ' ',
 'g',
 '_',
 'S',
 '8',
 'j',
 '`',
 'b',
 'd',
 ')',
 't',
 '2',
 'B',
 '9')

In [15]:
char2int

{'h': 0,
 'i': 1,
 'u': 2,
 ';': 3,
 'a': 4,
 'P': 5,
 'y': 6,
 'G': 7,
 'r': 8,
 'T': 9,
 'x': 10,
 'Y': 11,
 'w': 12,
 'Z': 13,
 '0': 14,
 'A': 15,
 '7': 16,
 'J': 17,
 'H': 18,
 '1': 19,
 '(': 20,
 '"': 21,
 'k': 22,
 'V': 23,
 'v': 24,
 'C': 25,
 'o': 26,
 ',': 27,
 'D': 28,
 'M': 29,
 '&': 30,
 '/': 31,
 'f': 32,
 '*': 33,
 'K': 34,
 'c': 35,
 'p': 36,
 '!': 37,
 'F': 38,
 'm': 39,
 'l': 40,
 'W': 41,
 'E': 42,
 'L': 43,
 '@': 44,
 'X': 45,
 's': 46,
 'O': 47,
 '\n': 48,
 'Q': 49,
 ':': 50,
 '5': 51,
 'N': 52,
 'I': 53,
 'n': 54,
 '-': 55,
 '3': 56,
 '%': 57,
 '.': 58,
 '?': 59,
 'U': 60,
 'q': 61,
 'e': 62,
 '$': 63,
 'R': 64,
 '4': 65,
 "'": 66,
 'z': 67,
 '6': 68,
 ' ': 69,
 'g': 70,
 '_': 71,
 'S': 72,
 '8': 73,
 'j': 74,
 '`': 75,
 'b': 76,
 'd': 77,
 ')': 78,
 't': 79,
 '2': 80,
 'B': 81,
 '9': 82}

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an intgere (via our created dictionary) and *then* converted into a column vector where only it's corresponsing integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [16]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype = np.float32)

    # Fill the appropriate elements with one
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1

    # Reshape it to get back the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `n_seqs` (also refered to as "batch size" in other places). Each of those sequences will be `n_steps` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ sequences. **

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [17]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''

    # Get the number of characters per batch
    batch_size = n_seqs * n_steps

    ## TODO: Get the number of batches we can make
    n_batches = len(arr) // batch_size

    ## TODO: Keep only enough characters to make full batches
    arr = arr[0: n_seqs * n_steps * n_batches]

    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((n_seqs, -1))

    ## TODO: Make batches
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


In [18]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [19]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[25  0  4 36 79 62  8 69 19 48]
 [69  4 39 69 54 26 79 69 70 26]
 [24  1 54 58 48 48 21 11 62 46]
 [54 69 77  2  8  1 54 70 69  0]
 [69  1 79 69  1 46 27 69 46  1]
 [69 53 79 69 12  4 46 48 26 54]
 [ 0 62 54 69 35 26 39 62 69 32]
 [ 3 69 76  2 79 69 54 26 12 69]
 [79 69  1 46 54 66 79 58 69  9]
 [69 46  4  1 77 69 79 26 69  0]]

y
 [[ 0  4 36 79 62  8 69 19 48 48]
 [ 4 39 69 54 26 79 69 70 26  1]
 [ 1 54 58 48 48 21 11 62 46 27]
 [69 77  2  8  1 54 70 69  0  1]
 [ 1 79 69  1 46 27 69 46  1  8]
 [53 79 69 12  4 46 48 26 54 40]
 [62 54 69 35 26 39 62 69 32 26]
 [69 76  2 79 69 54 26 12 69 46]
 [69  1 46 54 66 79 58 69  9  0]
 [46  4  1 77 69 79 26 69  0 62]]


If you implemented `get_batches` correctly, the above output should look something like
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

In [20]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[25  0  4 36 79 62  8 69 19 48]
 [69  4 39 69 54 26 79 69 70 26]
 [24  1 54 58 48 48 21 11 62 46]
 [54 69 77  2  8  1 54 70 69  0]
 [69  1 79 69  1 46 27 69 46  1]
 [69 53 79 69 12  4 46 48 26 54]
 [ 0 62 54 69 35 26 39 62 69 32]
 [ 3 69 76  2 79 69 54 26 12 69]
 [79 69  1 46 54 66 79 58 69  9]
 [69 46  4  1 77 69 79 26 69  0]]

y
 [[ 0  4 36 79 62  8 69 19 48 48]
 [ 4 39 69 54 26 79 69 70 26  1]
 [ 1 54 58 48 48 21 11 62 46 27]
 [69 77  2  8  1 54 70 69  0  1]
 [ 1 79 69  1 46 27 69 46  1  8]
 [53 79 69 12  4 46 48 26 54 40]
 [62 54 69 35 26 39 62 69 32 26]
 [69 76  2 79 69 54 26 12 69 46]
 [69  1 46 54 66 79 58 69  9  0]
 [46  4  1 77 69 79 26 69  0 62]]


---
## Defining the network with PyTorch

Below is where you'll define the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic LSTM cell as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial cell state of all zeros. This is done like so

```python
self.init_weights()
```

In [21]:
class CharRNN(nn.Module):

    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM, self.lstm
        self.lstm = nn.LSTM(len(tokens), n_hidden, n_layers, dropout=drop_prob, batch_first=True)

        ## TODO: define a dropout layer, self.dropout
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer, self.fc
        self.fc = nn.Linear(n_hidden,len(tokens))

        # initialize the weights
        self.init_weights()


    def forward(self, x, hc):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hc`. '''

        ## TODO: Get x, and the new hidden state (h, c) from the lstm
        x, (h,c) = self.lstm(x,hc)

        ## TODO: pass x through a droupout layer
        x = self.dropout(x)

        # Stack up LSTM outputs using view
        x = x.reshape(x.size()[0]*x.size()[1], self.n_hidden)

        ## TODO: put x through the fully-connected layer
        x = self.fc(x)

        # return x and the hidden state (h, c)
        return x, (h, c)


    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.

            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.to(device)
        else:
            self.cpu()

        if h is None:
            h = self.init_hidden(1)

        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        inputs = torch.from_numpy(x)
        if cuda:
            inputs = inputs.to(device)

        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        if cuda:
            p = p.cpu()

        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        return self.int2char[char], h

    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1

        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)

    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())


### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

In [22]:
# antrenare pe GPU daca exista

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [23]:
if 'net' in locals():
    del net

In [24]:
# define and print the net
net = CharRNN(chars, n_hidden=512, n_layers=2)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [25]:
opt = torch.optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [26]:
## ---- keep notebook from crashing during training --- ##
import time

def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network

        Arguments
        ---------

        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss

    '''

    net.train()
    # opt = torch.optim.Adam(net.parameters(), lr=lr)
    # criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if cuda:
        net.to(device)

    counter = 0
    n_chars = len(net.chars)

    old_time = time.time()

    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        for x, y in get_batches(data, n_seqs, n_steps):

            if time.time() - old_time > 60:
                old_time = time.time()

            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if cuda:
                inputs, targets = inputs.to(device), targets.to(device)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()

            output, h = net.forward(inputs, h)
            targets = targets.type(torch.LongTensor)
            loss = criterion(output.to(device), targets.view(n_seqs*n_steps).to(device))

            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()

            if counter % print_every == 0:

                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.to(device), targets.to(device)

                    targets = targets.type(torch.LongTensor)

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output.to(device), targets.view(n_seqs*n_steps).to(device))

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Time to train

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes (number of sequences and number of steps), and start the training. With the train function, we can set the number of epochs, the learning rate, and other parameters. Also, we can run the training on a GPU by setting `cuda=True`.

In [27]:
# if 'net' in locals():
#     del net

In [28]:
# # define and print the net
# net = CharRNN(chars, n_hidden=512, n_layers=2)
# print(net)

In [29]:
torch.cuda.empty_cache()

n_seqs, n_steps = 128, 300

# you may change cuda to True if you plan on using a GPU!
# also, if you do, please INCREASE the epochs to 25

# Open the training log file.
log_file = 'training_log.txt'
f = open(log_file, 'w')

# TRAIN
train(net, encoded, epochs=80, n_seqs=n_seqs, n_steps=n_steps, lr=0.01, cuda=True, print_every=10)

# Close the training log file.
f.close()


Epoch: 1/80... Step: 10... Loss: 3.9137... Val Loss: 3.8650
Epoch: 1/80... Step: 20... Loss: 3.5443... Val Loss: 3.5136
Epoch: 1/80... Step: 30... Loss: 3.4020... Val Loss: 3.4028
Epoch: 1/80... Step: 40... Loss: 3.3388... Val Loss: 3.3452
Epoch: 2/80... Step: 50... Loss: 3.3013... Val Loss: 3.3125
Epoch: 2/80... Step: 60... Loss: 3.2767... Val Loss: 3.2887
Epoch: 2/80... Step: 70... Loss: 3.2716... Val Loss: 3.2706
Epoch: 2/80... Step: 80... Loss: 3.2658... Val Loss: 3.2542
Epoch: 2/80... Step: 90... Loss: 3.2323... Val Loss: 3.2401
Epoch: 3/80... Step: 100... Loss: 3.2216... Val Loss: 3.2231
Epoch: 3/80... Step: 110... Loss: 3.1983... Val Loss: 3.2073
Epoch: 3/80... Step: 120... Loss: 3.1755... Val Loss: 3.1882
Epoch: 3/80... Step: 130... Loss: 3.1772... Val Loss: 3.1684
Epoch: 4/80... Step: 140... Loss: 3.1462... Val Loss: 3.1451
Epoch: 4/80... Step: 150... Loss: 3.1060... Val Loss: 3.1217
Epoch: 4/80... Step: 160... Loss: 3.0962... Val Loss: 3.0902
Epoch: 4/80... Step: 170... Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `n_seqs` - Number of sequences running through the network in one pass.
* `n_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `n_seqs` - Number of sequences running through the network in one pass.
* `n_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [30]:
# change the name, for saving multiple files
model_name = 'rnn_1_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

Now that the model is trained, we'll want to sample from it. To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### Top K sampling

Our predictions come from a categorcial probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [31]:
def sample(net, size, prime='The', top_k=None, cuda=False):

    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [32]:
print(sample(net, 2000, prime='Anna', top_k=5, cuda=False))

Annas to a mome himses, and and allowed the mintter of all her the
waser he had thing were the consconacion of his with themething and
she helped, but strings, whome he was to she heard his heart the him,
and she had her tolk a some timest a consion of that herese a dispore on
the sand the could befieving him to be and that had been to anding
them, and andersting the moried the moth that the pleess on he
cares and was to antwe to the carrat whene the milet ond weally though to
his said, had to he was soring as it all them. The corlice at in the
conserating hands, at allow himees and alother that that sterng the
could an this workes to he had and she to be her, he had not stepling the
same of her this to the parrait of her have and and the with her some,
and stood herss of she was a speasa of all other to as she had becale
the pationant with thit the presticat were has but that the seep as he
could now to she she saw that the sarrighen soriget and somition of
her, and she said of she ha

In [33]:
# Here we have loaded in a model that trained over 1 epoch `rnn_1_epoch.net`
with open('rnn_1_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [34]:
# Change cuda to True if you are using GPU!
print(sample(loaded, 2000, cuda=True, top_k=5, prime="And Levin said"))

And Levin said than sather her tushing out to batk the his must
his she teemed and which his hand him and with she was the sittres, the
crommored. And this stind was still at the pase, as he would be
the stopt of indiraritable, and when his forther heard, there he was
a conder to she was so her tood andone the tomething a mettre of all
the seare. "Ahe same as a lithle an the sore ounds to the part if."

"And where, said he heart her," said Levin, lotking his him, who had
and have been all to at the propsted...."

This with a was told of his sawer at when semether had heart the coult of
her.

She was sond the crover her they her had the have out, be to the came
with the samill, a sate to hall and went the porstion, and to see the
conderteded--a came to bring that the shallist on he fact. The heard
a seepicully say the comsions a struen a conctace. And her aldays, they he
would not that it talk to the chomply and and a mary at word at the
collance was somethancs in whoch he had not to ha